In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'pf-henry-405314' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

# Librerias

In [ ]:
import pandas as pd

import datetime

from google.cloud import storage
from google.cloud import bigquery
import pandas
import pytz

from google.colab import drive
from google.colab import auth

from pandas import json_normalize

from textblob import TextBlob


# Método directo de Drive

# New Mexico, California, Arizona, Alabama, Arkansas, Florida, Georgia, Louisiana, Mississippi, North Carolina, Oklahoma, South Carolina, Texas, Tennessee

In [ ]:
# se extraen los archivos de drive, se precesan y se cargan a BigQuery (este metodo es para validacion)

def clean_dataframe(df):
    """ funcion de pre-procesamiento que se aplica a cada dataframe creado a partir de la consulta en BigQuery"""

    df.drop(['pics'], axis=1, inplace=True)
    df['resp'] = df['resp'].fillna(0)
    df['resp'] = df['resp'].where(df['resp'] == 0, 1)
    df['user_id'] = df['user_id'].astype(str)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    df['user_id'] = df['user_id'].str.replace('.', '', regex=True).str.replace('e\+20', '', regex=True)
    df['time'] = pd.to_datetime(df['time'], unit='ms')
    df['state'] = 'Alabama'
    df = df[df['time'].dt.year > 2017]

# Montar el drive
drive.mount('/content/drive')

# Acceder a la carpeta compartida
folder_path = "/content/drive/My Drive/Google Maps/reviews-estados/review-Alabama/" # para cargar los reviews de otro estado solo se cambia el nombre del estado

# Crear un diccionario para almacenar los dataframes
dataframes = []

# Leer y limpiar cada archivo .json
for i in range(1,13):  #rango según la cantidad de archivos .json
    filename = str(i) + '.json'
    file_path = folder_path + filename
    data = pd.read_json(file_path, lines=True)
    clean_dataframe(data)
    dataframes.append(data)

# Concatenar los dataframes en uno solo
result = pd.concat(dataframes, ignore_index=True)

Mounted at /content/drive


In [ ]:
# se cargan en bigquery las tablas generadas estado por estado

auth.authenticate_user()
print('Authenticated')

# Especificar el proyecto y el dataset de BigQuery
project_id = 'pf-henry-405314'

table_id = 'pf-henry-405314.Google.Alabama'

# Crear un cliente de BigQuery
client = bigquery.Client(project=project_id)

# Cargar el DataFrame a BigQuery
job_config =  bigquery.LoadJobConfig( schema =  [
                bigquery.SchemaField('gmap_id',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('rating',	bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField('text',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('resp',	bigquery.enums.SqlTypeNames.INTEGER),
                bigquery.SchemaField('time',	bigquery.enums.SqlTypeNames.DATETIME),
                bigquery.SchemaField('name',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('user_id',	bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField('state',	bigquery.enums.SqlTypeNames.STRING)
                ],
              write_disposition="WRITE_TRUNCATE",)

job = client.load_table_from_dataframe(result, table_id, job_config=job_config)  # llamada a la API

job.result()  # Espera mientras se ejecuta el trabajo

table = client.get_table(table_id)  # llamada a la API

print("Cargadas {} filas y {} columnas a {}".format(table.num_rows, len(table.schema), table_id))